## 인공지능과제 14 - NN
NN 모듈 필요 단계
1. 데이터 로드
2. 필요 없는 Column을 drop
3. Mileage, Engine, Power등의 데이터에서 CC등의 단위를 제거하여, 숫자
데이터로 바꿔줌
4. 남은 문자열 Column들을 인코딩
5. 알고싶은 목표인 Price Column과 그 외 Column 분리
6. StandardScaler, MinMaxScaler등 알맞은 Scaler 선택
7. nearest_neighbors의 수를 선택
8. 맨하탄 거리, 유클리디언 거리등 이웃간의 거리 측정방법 선택


In [2]:
from __future__ import print_function
import os

from io import StringIO
from IPython.display import Image, display

from sklearn.tree import export_graphviz
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

#from sklearnex import patch_sklearn
#patch_sklearn()

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('train.csv')

In [6]:
data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [7]:
print(data.dtypes)

Name                  object
Location              object
Year                   int64
Kilometers_Driven      int64
Fuel_Type             object
Transmission          object
Owner_Type            object
Mileage               object
Engine                object
Power                 object
Seats                float64
New_Price             object
Price                float64
dtype: object


In [8]:
train, test = train_test_split(data, test_size=0.3, random_state=42)
train_data = train.drop(labels="New_Price", axis = 1)
test = test.drop(labels="New_Price", axis = 1)

In [9]:
train_data.describe()

,Year,Kilometers_Driven,Seats,Price
count,4213.000000,4213.000000,4185.000000,4213.000000
mean,2013.396392,57529.234987,5.277419,9.466622
std,3.237586,38099.559922,0.805531,11.229414
min,1998.000000,600.000000,0.000000,0.440000
25%,2011.000000,34000.000000,5.000000,3.500000
50%,2014.000000,52827.000000,5.000000,5.680000
75%,2016.000000,72100.000000,5.000000,9.900000
max,2019.000000,775000.000000,10.000000,160.000000


In [10]:
train_data = train_data[train_data['Mileage'].notna()]
print(train_data.shape)
train_data = train_data[train_data['Engine'].notna()]
print(train_data.shape)
train_data = train_data[train_data['Power'].notna()]
print(train_data.shape)
train_data = train_data[train_data['Seats'].notna()]
print(train_data.shape)

(4212, 12)
(4188, 12)
(4188, 12)
(4184, 12)


In [11]:
print(train_data['Location'].unique())
print(train_data['Fuel_Type'].unique())
print(train_data['Transmission'].unique())
print(train_data['Owner_Type'].unique())

['Jaipur' 'Hyderabad' 'Kochi' 'Kolkata' 'Ahmedabad' 'Mumbai' 'Coimbatore'
 'Chennai' 'Bangalore' 'Delhi' 'Pune']
['Diesel' 'Petrol' 'CNG' 'LPG']
['Manual' 'Automatic']
['First' 'Second' 'Third' 'Fourth & Above']


In [12]:
train_data.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

In [13]:
train_data = train_data.reset_index(drop=True)

In [14]:
for i in range(train_data.shape[0]):
    train_data.at[i, 'Company'] = train_data['Name'][i].split()[0]
    train_data.at[i, 'Mileage(km/kg)'] = train_data['Mileage'][i].split()[0]
    train_data.at[i, 'Engine(CC)'] = train_data['Engine'][i].split()[0]
    train_data.at[i, 'Power(bhp)'] = train_data['Power'][i].split()[0]


In [15]:
x = 'n'
count = 0
position = []
for i in range(train_data.shape[0]):
    if train_data['Power(bhp)'][i]=='null':
        x = 'Y'
        count = count + 1
        position.append(i)
print(x)
print(count)
print(position)

Y
66
[167, 201, 337, 376, 392, 448, 505, 554, 591, 600, 641, 692, 710, 752, 823, 879, 917, 920, 935, 1069, 1120, 1208, 1283, 1288, 1351, 1454, 1546, 1738, 1821, 1835, 1837, 1938, 2087, 2089, 2133, 2140, 2219, 2249, 2257, 2271, 2315, 2425, 2509, 2551, 2648, 2704, 2713, 2826, 2845, 2979, 3046, 3103, 3141, 3151, 3158, 3189, 3203, 3208, 3354, 3570, 3605, 3924, 3933, 4004, 4064, 4067]


In [16]:
train_data = train_data.drop(train_data.index[position])
train_data = train_data.reset_index(drop=True)

In [17]:
train_data['Mileage(km/kg)'] = train_data['Mileage(km/kg)'].astype(float)
train_data['Engine(CC)'] = train_data['Engine(CC)'].astype(float)
train_data['Power(bhp)'] = train_data['Power(bhp)'].astype(float)
train_data['Year'] = train_data['Year'].astype(float)
train_data['Kilometers_Driven'] = train_data['Kilometers_Driven'].astype(float)


In [18]:
train_data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company,Mileage(km/kg),Engine(CC),Power(bhp)
0,Volkswagen Polo Diesel Trendline 1.2L,Jaipur,2011.0,77000.0,Diesel,Manual,First,22.07 kmpl,1199 CC,73.9 bhp,5.0,2.90,Volkswagen,22.07,1199.0,73.9
1,Hyundai i10 Sportz,Hyderabad,2016.0,19947.0,Petrol,Manual,First,20.36 kmpl,1197 CC,78.9 bhp,5.0,4.50,Hyundai,20.36,1197.0,78.9
2,Mahindra XUV500 W8 2WD,Kochi,2014.0,70963.0,Diesel,Manual,First,15.1 kmpl,2179 CC,140 bhp,7.0,7.83,Mahindra,15.10,2179.0,140.0
3,Maruti Swift VDI BSIV,Kolkata,2013.0,115195.0,Diesel,Manual,First,25.2 kmpl,1248 CC,74 bhp,5.0,3.85,Maruti,25.20,1248.0,74.0
4,Volkswagen Polo Petrol Trendline 1.2L,Kochi,2011.0,58752.0,Petrol,Manual,First,16.47 kmpl,1198 CC,73.9 bhp,5.0,3.32,Volkswagen,16.47,1198.0,73.9


In [19]:
train_data.shape

(4118, 16)

In [20]:
train_data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company,Mileage(km/kg),Engine(CC),Power(bhp)
0,Volkswagen Polo Diesel Trendline 1.2L,Jaipur,2011.0,77000.0,Diesel,Manual,First,22.07 kmpl,1199 CC,73.9 bhp,5.0,2.90,Volkswagen,22.07,1199.0,73.9
1,Hyundai i10 Sportz,Hyderabad,2016.0,19947.0,Petrol,Manual,First,20.36 kmpl,1197 CC,78.9 bhp,5.0,4.50,Hyundai,20.36,1197.0,78.9
2,Mahindra XUV500 W8 2WD,Kochi,2014.0,70963.0,Diesel,Manual,First,15.1 kmpl,2179 CC,140 bhp,7.0,7.83,Mahindra,15.10,2179.0,140.0
3,Maruti Swift VDI BSIV,Kolkata,2013.0,115195.0,Diesel,Manual,First,25.2 kmpl,1248 CC,74 bhp,5.0,3.85,Maruti,25.20,1248.0,74.0
4,Volkswagen Polo Petrol Trendline 1.2L,Kochi,2011.0,58752.0,Petrol,Manual,First,16.47 kmpl,1198 CC,73.9 bhp,5.0,3.32,Volkswagen,16.47,1198.0,73.9


In [21]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()
ohc = OneHotEncoder()

data_label = train_data.copy()

label_cols = ['Fuel_Type', 'Transmission', 'Owner_Type', 'Company']
ohc_cols = ['Location']

#레이블 인코딩
for col in label_cols:
    # Integer encode the string categories
    dat = le.fit_transform(data_label[col]).astype(int)
    col_names = [col]
    data_label = data_label.drop(col, axis=1)
    
    new_df = pd.DataFrame(dat, 
                          index=data_label.index, 
                          columns=col_names)
    data_label = pd.concat([data_label, new_df], axis=1)


#원 핫 인코딩 적용
for col in ohc_cols:
    
    # Integer encode the string categories
    # 레이블 인코딩 적용
    dat = le.fit_transform(data_label[col]).astype(int)
    
    # Remove the original column from the dataframe
    # 데이터에서 원래 컬럼 제거
    data_label = data_label.drop(col, axis=1)

    # One hot encode the data--this returns a sparse array
    # 원핫 인코딩
    new_dat = ohc.fit_transform(dat.reshape(-1,1))

    # Create unique column names
    # 새로운 컬럼 이름 설정
    n_cols = new_dat.shape[1]
    col_names = ['_'.join([col, str(x)]) for x in range(n_cols)]

    # Create the new dataframe
    new_df = pd.DataFrame(new_dat.toarray(), 
                          index=data_label.index, 
                          columns=col_names)

    # Append the new data to the dataframe
    data_label = pd.concat([data_label, new_df], axis=1)

train_data = data_label.copy()
train_data.head()

,Name,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Mileage(km/kg),Engine(CC),...,Location_1,Location_2,Location_3,Location_4,Location_5,Location_6,Location_7,Location_8,Location_9,Location_10
0,Volkswagen Polo Diesel Trendline 1.2L,2011.0,77000.0,22.07 kmpl,1199 CC,73.9 bhp,5.0,2.90,22.07,1199.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Hyundai i10 Sportz,2016.0,19947.0,20.36 kmpl,1197 CC,78.9 bhp,5.0,4.50,20.36,1197.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Mahindra XUV500 W8 2WD,2014.0,70963.0,15.1 kmpl,2179 CC,140 bhp,7.0,7.83,15.10,2179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Maruti Swift VDI BSIV,2013.0,115195.0,25.2 kmpl,1248 CC,74 bhp,5.0,3.85,25.20,1248.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Volkswagen Polo Petrol Trendline 1.2L,2011.0,58752.0,16.47 kmpl,1198 CC,73.9 bhp,5.0,3.32,16.47,1198.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [22]:
train_data.drop(["Name"],axis=1,inplace=True)
train_data.drop(["Mileage"],axis=1,inplace=True)
train_data.drop(["Engine"],axis=1,inplace=True)
train_data.drop(["Power"],axis=1,inplace=True)
#train_data.drop(["Location"],axis=1,inplace=True)
#train_data.drop(["Fuel_Type"],axis=1,inplace=True)
#train_data.drop(["Transmission"],axis=1,inplace=True)
#train_data.drop(["Owner_Type"],axis=1,inplace=True)
#train_data.drop(["Company"],axis=1,inplace=True)
#여기 float?
#train_data["Price"] = train_data["Price"].astype(float)
train_data["Price"] = train_data["Price"].astype(int)





In [23]:
train_data.head()

,Year,Kilometers_Driven,Seats,Price,Mileage(km/kg),Engine(CC),Power(bhp),Fuel_Type,Transmission,Owner_Type,...,Location_1,Location_2,Location_3,Location_4,Location_5,Location_6,Location_7,Location_8,Location_9,Location_10
0,2011.0,77000.0,5.0,2,22.07,1199.0,73.9,1,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2016.0,19947.0,5.0,4,20.36,1197.0,78.9,3,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2014.0,70963.0,7.0,7,15.10,2179.0,140.0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2013.0,115195.0,5.0,3,25.20,1248.0,74.0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2011.0,58752.0,5.0,3,16.47,1198.0,73.9,3,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [24]:
train, test = train_test_split(train_data, random_state=42, test_size=0.3)


In [25]:
from sklearn import preprocessing


y_train = train.Price
train_x = train.drop(labels="Price", axis = 1)
y_test = test.Price
test_x = test.drop(labels="Price", axis = 1)

lab = preprocessing.LabelEncoder()

y_data = lab.fit_transform(y_train)
#y_data = lab.fit_transform(train['Price'])

print(y_data)


[ 2 17 29 ...  0 37  3]


In [26]:
# Mute the sklearn warning
import warnings
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler    #StandardScaler 비교를 위해 추가

warnings.filterwarnings('ignore', module='sklearn')

#msc = MinMaxScaler()
msc = StandardScaler()
#msc = MaxAbsScaler()

train_x = pd.DataFrame(msc.fit_transform(train_x),  # this is an np.array, not a dataframe.
                    columns=train_x.columns)

test_x = pd.DataFrame(msc.fit_transform(test_x),  # this is an np.array, not a dataframe.
                    columns=test_x.columns)

In [27]:
train_x.head()

,Year,Kilometers_Driven,Seats,Mileage(km/kg),Engine(CC),Power(bhp),Fuel_Type,Transmission,Owner_Type,Company,...,Location_1,Location_2,Location_3,Location_4,Location_5,Location_6,Location_7,Location_8,Location_9,Location_10
0,-1.122877,0.046666,-0.353598,-0.615933,-0.710362,-0.804420,1.092183,0.638143,2.021123,-0.950489,...,-0.263416,-0.294494,-0.345287,3.116690,-0.369978,-0.27381,-0.352794,-0.312229,-0.388534,-0.330628
1,0.160411,0.998702,2.116874,-1.315693,2.248334,1.054598,-0.891227,0.638143,-0.459131,1.521460,...,-0.263416,-0.294494,-0.345287,-0.320853,-0.369978,-0.27381,-0.352794,-0.312229,2.573779,-0.330628
2,0.802055,-0.262083,-0.353598,-1.212449,0.858443,1.711826,-0.891227,-1.567047,-0.459131,0.480639,...,-0.263416,-0.294494,-0.345287,-0.320853,-0.369978,-0.27381,2.834516,-0.312229,-0.388534,-0.330628
3,0.802055,-0.764620,-0.353598,-0.416330,-0.707049,-0.731185,1.092183,0.638143,-0.459131,1.651563,...,-0.263416,-0.294494,2.896139,-0.320853,-0.369978,-0.27381,-0.352794,-0.312229,-0.388534,-0.330628
4,-0.160411,-0.487123,-0.353598,-0.416330,-0.707049,-0.731185,1.092183,0.638143,-0.459131,1.651563,...,-0.263416,-0.294494,-0.345287,-0.320853,-0.369978,-0.27381,-0.352794,3.202782,-0.388534,-0.330628


In [28]:
y_train.head()

1588     2
997     17
3127    29
1952     6
1132     2
Name: Price, dtype: int32

In [29]:
train_x.shape
train_x.dtypes

Year                 float64
Kilometers_Driven    float64
Seats                float64
Mileage(km/kg)       float64
Engine(CC)           float64
Power(bhp)           float64
Fuel_Type            float64
Transmission         float64
Owner_Type           float64
Company              float64
Location_0           float64
Location_1           float64
Location_2           float64
Location_3           float64
Location_4           float64
Location_5           float64
Location_6           float64
Location_7           float64
Location_8           float64
Location_9           float64
Location_10          float64
dtype: object

In [52]:
from sklearn.neighbors import KNeighborsClassifier
def accuracy(real, predict):
    return sum(real == predict) / float(real.shape[0])  #원본은 오타라 조교님이 수정하심


knn = KNeighborsClassifier(n_neighbors=1, p=1)  # k=1일때가 제일 좋음, 맨하탄 거리 < 유클리디언 거리

knn = knn.fit(train_x, y_train)

y_pred = knn.predict(train_x)

#y_data = lab.fit_transform(y_train)

score = accuracy(y_data, y_pred)

print(score)
print(y_pred)


0.9819569743233866
[ 2 17 29 ...  0 37  3]


In [53]:

from sklearn.metrics import mean_squared_error

mean_squared_error(y_test,knn.predict(test_x))

33.27993527508091

In [54]:
print("Rmse : ", mean_squared_error(y_test,knn.predict(test_x))**0.5)
print("Accuracy on Traing set: ",knn.score(train_x,y_train))
print("Accuracy on Testing set: ",knn.score(test_x,y_test))

Rmse :  5.768876430907574
Accuracy on Traing set:  0.998265093684941
Accuracy on Testing set:  0.2872168284789644


### NN 결과 정리

##### 1. 우선 위 코드에서 NN 알고리즘의 올바른 적용을 위한 모든 프로세스 단계(절차)를 이행하였고, 도출된 실험 결과를 정량적 비교 및 분석을 수행하여 최적 기법을 도출하였다.

1) 데이터 전처리 : 필요 없는 Column을 drop하였고, Mileage, Engine, Power등의 데이터에서 CC등의 단위를 제거하여, 숫자 데이터로 바꿔준 후, 남은 문자열 Column들을 인코딩하였다.

2) 인코딩 방법 : Fuel_Type, Transmission, Owner_Type, Company 이 4개의 Column은 순서가 필요하다고 판단하였기에 OneHotEncoder로, Location은 그렇지 않다고 판단하여 LabelEncoder로 인코딩하였다. <br/>
실제로 이렇게 인코딩을 하였을 때, 조금이지만 MSE가 가장 낮게 나왔다.

3) Scaler 선택 : 선택에 따라 가장 큰 차이가 나는 부분이였다. StandardScaler가 가장 낮은 MSE를 보여주었다. 데이터에 이상치나 튀는 값이 없기 때문에 일반적인 StandardScaler를 활용했을 때, 가장 좋은 결과를 이끌어낼 수 있었다.<br/>
Non_Scaled = 68.02..
MinMaxScaler = 44.13..
StandardScaler = 33.27..
MaxAbsScaler = 69.05..

4) nearest_neighbors의 수 : 12주차 과제에서 배웠던 대로 비교 축이 2개, 즉 짝수개이기 때문에 k=1일때가 accuracy가 가장 높아, MSE가 가장 낮게 나왔다.<br/>
k=1 : 33.27.., k=2 : 36.06.., k=3 : 46.63.. ...

5) 거리 측정방법 선택 : 값들의 분포가 각각의 값마다 많이 다르기때문에, 맨하탄 거리를 활용했을 때가 유클리디언 거리를 활용했을 때보다 MSE가 소폭 낮게 나왔다.<br/>
33.27.. vs 35.94..

최선의 결과를 낼 수 있는 방법들을 선택한 결과, MSE = 33.27993527508091, RMSE = 5.768876430907574가 나왔다.

##### 2. 최적 기법으로 선정된 알고리즘의 우월성 주장(Claim)를 정성적으로 명확히 설명(Theoretical Evidence)하고, 팀이 목적으로 한 바를 종합적으로 Reasoning(concluding remarks)하였다.

NN의 결과<br/>
Rmse :  5.768876430907574<br/>
Accuracy on Traing set:  0.998265093684941<br/>
Accuracy on Testing set:  0.2872168284789644

NB의 결과
Rmse :  11.488761209666533<br/>
Accuracy on Traing set:  0.1999043519846963<br/>
Accuracy on Testing set:  0.18460680423870607

SVM의 결과<br/>
Rmse :  1044.9557155198318<br/>
Accuracy on Traing set:  0.23023973415618323<br/>
Accuracy on Testing set:  0.21373200442967885

DT의 결과<br/>
Rmse :  3.7793213100801744<br/>
Accuracy on Traing set:  0.9757482225204354<br/>
Accuracy on Testing set:  0.8697921871113656
<br/><br/>

NN의 경우는 많은 특성을 처리하는 능력이 부족하다는 단점이 있는데, 이 데이터의 경우는 특성이 많지 않아 꽤나 좋은 결과를 도출해냈다.<br/>
다만, 노이즈 데이터에 예민하다는 특성때문에 DT에 비해서는 안좋은 결과가 나온 듯 하다. 그래프를 그려본 결과 이러한 노이즈 데이터가 꽤나 존재했기 때문이다.<br/>
k=1로 하여 그 영향을 줄여보고자 했으나, 분명한 한계가 존재 했다.


NB의 경우는 모든 변수가 독립변수라는 가설하에 작동한다는 특징이 존재한다. 그렇기에 데이터가 가설과 다를 경우 정확도가 낮아진다.<br/>
이번 데이터가 그러한 경우인 듯 하다. 실제로 중고차 가격의 특성상, 다른 변수에 종속되는 변수가 여럿 존재했다.<br/>
또한 모든 숫자들을 int형로 변환해야하기 때문에 정확도가 낮게 나온 것으로 추측된다.<br/>
그렇기에 이번 데이터에선 좋은 결과가 나오지 않은 것 같다.

SVM의 경우는 해당 데이터의 target data가 실수형이기 때문에 적합하지 않다는 결론이 나왔다. <br/>
target data를 임의의 int데이터로 변환해야하므로, 원하는 데이터가 나오지 않을 수 있을 가능성이 크기 때문에, Rmse가 1000이 넘는 수치가 나왔다.<br/>
이에 대한 근거로, float data를 그대로 반영하였을 때, 오류율이 굉장히 크게 나왔다. (853127.4330011074)<br/>
또한, 결정경계선이 많이 겹칠 때 정확도가 낮아지는 SVM의 특성상, 값이 커질 수록 좋은 데이터도 있고 작아질 수록 좋은 데이터도 있는 우리 데이터는 더더욱 SVM과 맞지 않는다고 판단하였다.

결과적으로, 우리가 선택한 "Used Cars Price Prediction" 데이터에는 DT가 가장 우월한 알고리즘이라는 결과가 나오게 되었다. Rmse도 가장 낮고, Accuracy on Testing set도 가장 높은 수치를 보였다.<br/>
DT의 단점인 overfitting을 Random Forest를 사용하여 보완하였고, 우리 데이터가 차원이 엄청 높은 편이 아니기 때문에, 일반적인 경우에 가장 성능이 좋다고 알려진 DT가 우리 데이터에도 가장 잘 맞았다.<br/>
또한 각 특성이 개별적으로 처리되기 때문에 SVM에서는 단점이였던 우리 데이터의 특성들이 DT에서는 단점이 아니게 되었다.

다만, Accuracy on Traing set의 결과는 NN이 미약하게나마 더 높게 나왔는데, 이는 NN이 k=1로 설정하였기때문에, train처럼 데이터간의 거리가 작을 때에는 높은 Accuracy를 보이지만,<br/>
0.3배의 크기인 test에서는 데이터 간의 거리가 커져서 Accuracy가 낮아졌다고 판단하였다.
<br/><br/>

결론 : 우리가 선택한 "Used Cars Price Prediction" 데이터에는 DT가 가장 우월한 알고리즘이다.

